This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

In [1]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
# %pip install selenium chromedriver-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import boto3

In [5]:
# The file download processing section should be placed here
import time
from zipfile import ZipFile
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains

%pip install --upgrade --force-reinstall chromedriver-binary-auto
from dotenv import load_dotenv
import chromedriver_binary

DOWNLOAD_PATH = "https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing"
CURRENT_DIRECTORY = os.getcwd()
DOWLOAD_FILENAME = "data.zip"

def setupDriver(args: list[str], params: dict):
    print("Setting up webdriver...")
    options = Options()
    for arg in args:
        options.add_argument(arg)
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd('Page.setDownloadBehavior', params)
    print("Done!")
    return driver

args = ["--headless=True", "--window-size=1920,1080"]
params = {
    'downloadPath' : CURRENT_DIRECTORY,
    'behavior' : 'allow'
}
driver = setupDriver(args=args, params=params)
actions = ActionChains(driver)

# Hover the mouse to the download button
# Wait til 'Tải xuống' appear then click to download the file
driver.get(DOWNLOAD_PATH)
time.sleep(3)
popupDownloadButton = driver.find_element(By.XPATH, "//span[text()='Tải xuống']")
actions.move_to_element(popupDownloadButton).perform()
popupDownloadButton.click()
print(f"Downloading {DOWLOAD_FILENAME} to {CURRENT_DIRECTORY}...")

# Wait until the zip file has been downloaded
while DOWLOAD_FILENAME not in os.listdir(CURRENT_DIRECTORY):
    time.sleep(5)
print("Done!")

# Extract the data inside the zip file
with ZipFile(DOWLOAD_FILENAME, 'r') as zip_ref:
    print(f"Extracting {DOWLOAD_FILENAME}...")
    zip_ref.extractall(CURRENT_DIRECTORY)
    print(f"Done!")
# Go to the path
#...........................................................

  You can safely remove it manually.


  Using cached chromedriver_binary_auto-0.3.1-py3-none-any.whl
  Attempting uninstall: chromedriver-binary-auto
    Found existing installation: chromedriver-binary-auto 0.3.1
    Uninstalling chromedriver-binary-auto-0.3.1:
      Successfully uninstalled chromedriver-binary-auto-0.3.1
Note: you may need to restart the kernel to use updated packages.
Setting up webdriver...
Done!
Done!
Extracting data.zip...
Done!


In [6]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', '')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', '')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

In [7]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [8]:
# Upload file to minio
upload_file_to_minio('data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data3.json', 'data', 'data-raw/data3.json')